In [ ]:
import urllib.parse
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

# Load the Scrapped CSV

In [ ]:
df = pd.read_csv('dataset/pantarlih.csv')
df = df.drop(columns='Unnamed: 0')
df

In [ ]:
df['date'].unique()

In [ ]:
def str_timedelta_to_datetime(string):
    str_lower = string.lower();
    split = np.array(str_lower.split())
    result = None
    
    if (split[-1] == 'lalu'):
        for cat in ['detik', 'menit', 'jam', 'hari', 'minggu']:
            x = split[np.where(split == cat)[0] - 1]
            if(len(x) < 1):
                continue
            else:
                amount = int(x[0])
                if(cat == 'detik'):
                    deltaSubstract = pd.Timedelta(seconds=amount)
                elif(cat == 'menit'):
                    deltaSubstract = pd.Timedelta(minutes=amount)
                elif(cat == 'jam'):
                    deltaSubstract = pd.Timedelta(hours=amount)
                elif(cat == 'hari'):
                    deltaSubstract = pd.Timedelta(days=amount)
                elif(cat == 'minggu'):
                    deltaSubstract = pd.Timedelta(weeks=amount)

                substract_res = pd.date_range(datetime.now(), periods=1) - deltaSubstract
                result = substract_res[0].date()
                
    elif (split[-1] == 'ago'):
        for cat in ['seconds', 'minutes', 'hours', 'days', 'weeks']:
            x = split[np.where(split == cat)[0] - 1]
            if(len(x) < 1):
                continue
            else:
                amount = int(x[0])
                if(cat == 'seconds'):
                    deltaSubstract = pd.Timedelta(seconds=amount)
                elif(cat == 'minutes'):
                    deltaSubstract = pd.Timedelta(minutes=amount)
                elif(cat == 'hours'):
                    deltaSubstract = pd.Timedelta(hours=amount)
                elif(cat == 'days'):
                    deltaSubstract = pd.Timedelta(days=amount)
                elif(cat == 'weeks'):
                    deltaSubstract = pd.Timedelta(weeks=amount)

                substract_res = pd.date_range(datetime.now(), periods=1) - deltaSubstract
                result = substract_res[0].date()
                
    return result

In [ ]:
def translate_date(string):
    string = string.lower()
    month_list = np.array([
        'jan',
        'feb',
        'mar',
        'apr',
        'mei', 'may',
        'jun', 'june',
        'jul', 'july',
        'agu', 'aug',
        'sep', 'sept',
        'okt', 'oct',
        'nov',
        'des', 'dec',
        ])
    
    month_dict = {
        'jan' : 'jan',
        'feb' : 'feb',
        'mar' : 'mar',
        'apr' : 'apr',
        'may' : 'mei',
        'jun' : 'jun',
        'jul' : 'jul',
        'aug' : 'agu',
        'sep' : 'sep',
        'oct' : 'okt',
        'nov' : 'nov',
        'dec' : 'des',
    }
    
    month = "".join(re.split("[^a-zA-Z]*", string))
        
    if (len(np.where(month_list == month)[0]) > 0):
        
        str_res = string
        for index, key in month_dict.items():
            if (month == key):
                str_res = string.replace(key, index)
        return pd.to_datetime(str_res).date()
    else:
        return str_timedelta_to_datetime(string)

In [ ]:
for index, row in df.iterrows():
    val = df.loc[index, 'date']
    df.loc[index, 'date'] = translate_date(val)
df

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()
df.describe()

In [ ]:
df[['title']].groupby(df['date']).count()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df[['title']].groupby(df['date']).count())